In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
dataset = pd.read_csv('../input/allfeatures/features_all.csv')
X = dataset.iloc[: , :-1].values
y = dataset.iloc[:,-1].values

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(y).reshape(-1,1)).toarray()

In [ ]:
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 47, shuffle=True, test_size=0.15)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# making our data compatible to model.
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D

In [ ]:
model = Sequential()

model.add(Conv1D(16, 5,padding='same',input_shape=(162, 1), activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15))

model.add(Conv1D(32, 5,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15))

model.add(Conv1D(64, 5,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15))

model.add(Conv1D(128, 5,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15))

model.add(Conv1D(256, 5,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15))

model.add(Conv1D(512, 5,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15))

model.add(Conv1D(1024, 5,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.15)) 

model.add(Flatten())

model.add(Dense(256, activation ='relu'))
model.add(Dropout(0.15))

model.add(Dense(128, activation ='relu'))
model.add(Dropout(0.15))

model.add(Dense(64, activation ='relu'))

model.add(Dense(8, activation='relu'))

model.add(Dense(8, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=8, epochs=200, validation_data=(X_test, y_test))

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,y_test)[1]*100 , "%")

In [ ]:
pred_test = model.predict(X_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

In [ ]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(20)

In [ ]:
model.save('final_model_collectedSample.h5')